# 思路
* 1.爬取正确提交的题号，提交ID，提交代码
* 创建字典problem，存储题号:Str，提交ID:Str

## Setup

In [ ]:
!pip install -r .\requirements.txt

## 1.Problems

In [1]:
problems = {}

## 2.手动下载html文件，之后进行解析

In [ ]:
import scrapy
from lxml import etree

p = 1

while p <= 6:
    with open('p%s.html' % str(p), 'r', encoding='utf-8') as f:
        html = etree.HTML(f.read())
    nums = html.xpath("/html/body/center/center/center[1]/table/tbody/tr/td[3]/a")
    ids = html.xpath("/html/body/center/center/center[1]/table/tbody/tr/td[2]")
    stats = html.xpath("/html/body/center/center/center[1]/table/tbody/tr/td[4]/strong/span/font")
    for num, id in zip(nums, ids):
        problems[num.text] = id.text
    p += 1
print(problems)

## 3.使用cookie访问，获取代码，并存储在txt中

**获取cookies**

In [2]:
from time import sleep
import requests

cookies = {
    'PHPSESSID': '<value here>', # 会话ID
    'status_pp': '<value here>', # 状态
    'username': '<value here>', # 你的用户名
    'password': '<value here>', # MD5编码过的密码
    'status_sep': '<value here>', # 状态
}

cookiess = [
    {
        'name':'PHPSESSID',
        'value':'<value here>'
    },
    {
        'name':'status_pp',
        'value':'<value here>'
    },
    {
        'name':'username',
        'value':'<value here>'
    },
    {
        'name':'password',
        'value':'<value here>'
    },
    {
        'name':'status_sep',
        'value':'<value here>'
    }
]
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Cache-Control': 'max-age=0',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'PHPSESSID=5c1njff09bdmvoo8pn3s2hv1s1; status_pp=0; username=majiayin06; password=018355c4412a063980b7877698063fa9e5914bc8; status_sep=1660212756.3304',
    'Proxy-Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.47',
}

**爬取并存储** (这个不用运行)

In [ ]:
for key in problems.keys():
    params = {
        'runid': problems[key],
    }
    response = requests.get('http://py.ssoier.cn:8088/show_source.php', params=params, cookies=cookies, headers=headers, verify=False)
    html = etree.HTML(response.text)
    codes = html.xpath('''//*[@id="abc123"]''')
    for code in codes:
        with open("codes/%s.txt" % str(key), 'w+') as f:
            f.write(code.text)
            print(code.text)
    sleep(2)

## 4.自动提交

In [ ]:
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import re
from random import randint

def browser_init():
    options = Options()
    # options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
    browser = webdriver.Chrome(chrome_options = options, executable_path='./chromedriver.exe')
    browser.find_element
    return login(browser)

def login(browser):
    browser.get("http://py.ssoier.cn:8088/")
    for cooki in cookiess:
        browser.add_cookie(cooki)
    browser.refresh()
    return browser

def action(browser):
    for key in problems.keys():
        url = "http://py.ssoier.cn:8088/submit.php?pid=%s" % key
        browser.get(url)
        with open("codes/%s.txt" % key, "r") as f:
            browser.find_element("xpath", "/html/body/center/form/table/tbody/tr[3]/td/textarea").send_keys(
            '''%s''' % f.read()
            )#利用xpath查找元素进行输入文本
        browser.find_element("xpath", "/html/body/center/form/table/tbody/tr[3]/td/input[4]").click()
        sleep(40 + randint(0, 5))
if __name__ == "__main__":
    browser = browser_init()
    action(browser)